In [28]:
import json
from os import listdir
from os.path import join
import re

## Сбор данных

### по-одному документу 

In [23]:
topic = '«Память неподвластна времени»'

In [25]:
text = '''Кто сказал, что время лечит? Многие живут иллюзиями, надеясь на то, что
время излечит их раненую душу. Говорят, что со временем боль утихает. Но
это всего лишь кажется. Вся правда в том, что ты просто свыкаешься с этой
болью. А сердце всё помнит и украдкой сжимается от воспоминаний, не
позволяя тебе смириться. Воспоминания заставляют человека вернуться в
прошлое. Время проходит, но то, что осталось в памяти, стереть невозможно.
Данный вопрос не раз упоминался в художественной литературе.
Обратимся к рассказу И. А. Бунина «Тёмные аллеи», который доказывает,
что память неподвластна времени. Герои произведения предстают перед
нами совсем одинокими. Когда-то они испытывали друг к другу прекрасное
чувство. Но Николай Алексеевич предал Надежду, отказавшись от её любви.
С тех пор прошло много времени, и теперь каждый живёт своей жизнью. Но
любовь героини не умерла, а по-прежнему живёт в её сердце. Но несмотря на
долгие годы, прожитые в одиночестве одной лишь любовью, Наталья не
смогла забыть ту боль, которую предательски причинил ей тот, мыслям о
котором Наталья посвятила свою жизнь. Встретившись с возлюбленным
после долгой разлуки, героиня так и не смогла простить Николеньку, потому
что её любящее сердце навечно запомнило то, что так сильно оскорбило её.
В качестве второго аргумента хочу привести ещё один рассказ Ивана
Алексеевича «Холодная осень». В данном произведении основную часть
занимает осенний вечер, который оставил свой след в памяти рассказчицы.
Тот вечер запомнился ей последними словами, сказанными её
возлюбленным: «Ты поживи, порадуйся на свете, потом приходи ко мне…»
Героиня горько заплакала, убеждая его в том, что не переживёт его смерти.
Но, как оказалось, после его смерти жизнь была возможна. Спустя время
героиня многое пережила за свою жизнь, кажущуюся такой долгой, когда
перебираешь в памяти все прожитые годы. Вспоминая и задаваясь вопросом
о том, что же было в её жизни, героиня вдруг поняла, что все годы были
ненужным и непонятным сном. А тот самый холодный вечер – это всё, что
было в жизни рассказчицы.
В точение времени мы можем потерять что-то ценное или, что остаётся
только воспоминаниями. Но именно память – это единственная ценность,
которую невозможно потерять или стереть. И именно она заставляет
человека на миг вернуться в прошлое, каким бы оно ни было…
'''

In [100]:
def save_writing_to_json(path, writing):
    files = listdir(path)
    n = 0
    for file in files:
        r = re.compile('writing(\d+)\.json')
        new_n = int(r.match(file).group(1))
        if new_n - n > 1:
            break
        n = new_n
    n += 1
    name = 'writing' + str(n).zfill(6) + '.json'
    with open(join(path, name), 'w', encoding='utf-8') as output:
        json.dump(writing, output, indent=4, ensure_ascii=False)

In [54]:
path = 'data'

In [28]:
writing = {}
writing['topic'] = topic
writing['text'] = text
save_writing_to_json(path, writing)

### Парсинг сайтов

In [38]:
import requests
from bs4 import BeautifulSoup

In [96]:
page_url = 'https://sochinenie11.ru/primeri/'
writings_list = []

for i in range(8):
    i = i + 1
    response = requests.get(page_url)
    page_tree = BeautifulSoup(response.content, 'html.parser')
    subpages_url = [subpage.find('a')['href'] for subpage in page_tree.find_all('h2', {'class' : 'ssil'})]
    for sp_url in subpages_url:
        response = requests.get(sp_url)
        subpage_tree = BeautifulSoup(response.content, 'html.parser')
        if subpage_tree.find('div', {'class' : 'quote'}):
            pass
        elif subpage_tree.find('span', {'class' : 'attachment'}):
            pass
        else:
            topic = subpage_tree.find('h1', {'class' : 'ssil11'}).text
            text = subpage_tree.find('div', {'class' : 'otstup'}).text
            text = re.sub(r'Направление «.{1,30}»\.', '', text)
            writings_list.append({'topic':topic, 'text': text})
    if i != 8:
        page_url = page_tree.find('a', {'class':'button215'})['href']

In [97]:
len(writings_list)

43

In [101]:
for writing in writings_list:
    save_writing_to_json(path, writing)